In [3]:
DATASETS = "/data/datagouv/datagouv_all/datasets.csv"
DISCUSSIONS = "/data/datagouv/datagouv_all/discussions.csv"
ORGANIZATIONS = "/data/datagouv/datagouv_all/organizations.csv"
RESOURCES = "/data/datagouv/datagouv_all/resources.csv"
REUSES = "/data/datagouv/datagouv_all/reuses.csv"
USERS = "/data/datagouv/datagouv_all/users.csv"
TAGS = "/data/datagouv/datagouv_all/tags-2019-10-13-10-50.csv"
STOPWORDS_FR = [w.strip() for w in open("./stopwords_fr.txt").readlines()]

In [4]:
import pandas as pd
import numpy as np
from pprint import pprint

In [5]:
!pwd

/home/pavel/etalab/code/tags


In [6]:
class ObjectId:
    def __init__(self, id):
        self.id = id


In [7]:
dataset_df = pd.read_csv(DATASETS, sep=";", low_memory=False)

In [26]:
# Create a text corpus. A line per dataset combining the description and the title and the tags
dataset_df["all_text"] = dataset_df.description.fillna("") + " " +  dataset_df.title.fillna("") + " " + dataset_df.tags.fillna("[]").apply(lambda x: " ".join([f"__tag{t}" for t in eval(x)]))


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
tdf = TfidfVectorizer(ngram_range=(1,2), max_df=0.2, min_df=3, lowercase=True, stop_words=STOPWORDS_FR)
X = tdf.fit_transform(dataset_df["all_text"].values)
voc = np.array(tdf.get_feature_names())

In [33]:
def get_top_n_matrix(X, top_n=10):
    X = X.tolil()
    XX = X.copy()
    def max_n(row_data, row_indices, n):
            i = row_data.argsort()[-n:][::-1]
    #         print(i)
    #         print(row_data)
            top_values = row_data[i]
            top_indices = row_indices[i]  # do the sparse indices matter?
    #         print(top_indices)
            return top_values, top_indices, i

    for i in range(X.shape[0])[:]:
        top_values,top_indices=max_n(np.array(X.data[i]),np.array(X.rows[i]), top_n)[:2]
        XX.data[i]=top_values#.tolist()
        XX.rows[i]=top_indices#.tolist()
    return XX

def get_top_n_tokens(XX, index, voc):
    XX = XX.tocsr()
    top_ids = XX[index].nonzero()[1]
    pprint(list(zip(voc[top_ids], XX[index,top_ids].toarray()[0])))
    return voc[top_ids]

def get_input_text(dataset_df, index):
    pprint(dataset_df["all_text"].iloc[index])
def get_features(X, index, voc):
    foo = voc[X[index].nonzero()[1]]
    for f in X[index].nonzero()[1]:
        print(voc[f], "\t", X[index,f])
    

In [34]:
XX = get_top_n_matrix(X, 10)

In [35]:
target_id = 100

In [36]:
get_top_n_tokens(XX, target_id, voc);

[('évolution deux', 0.23211246911817562),
 ('médecine ville', 0.23211246911817562),
 ('évolution', 0.23048918875702593),
 ('remboursés assurance', 0.2278645172975488),
 ('molécule', 0.22427292520861608),
 ('montants remboursés', 0.21596267378185366),
 ('2017 chimio', 0.21171472196122684),
 ('années données', 0.21171472196122684),
 ('2014 2017', 0.20651564911578393),
 ('remboursés', 0.20501195129097466)]


In [37]:
get_input_text(dataset_df, target_id)

("Ce jeu de données présente l'évolution depuis 2014 des montants remboursés "
 "par l'assurance maladie par molécule en médecine de ville et l'évolution sur "
 'les deux dernières années- Données 2014-2017 Chimio 19 ')


In [38]:
dataset_df.iloc[target_id].description

"Ce jeu de données présente l'évolution depuis 2014 des montants remboursés par l'assurance maladie par molécule en médecine de ville et l'évolution sur les deux dernières années- Données 2014-2017"

In [39]:
get_features(X, target_id, voc)

19 	 0.12670768314004524
2014 	 0.16284014247278464
2014 2017 	 0.20651564911578393
2017 	 0.09154620777996295
2017 chimio 	 0.21171472196122684
années 	 0.11731125399796895
années données 	 0.21171472196122684
assurance 	 0.16231421492744486
assurance maladie 	 0.17062446635420728
chimio 	 0.1998128784455317
depuis 	 0.08940186993369925
depuis 2014 	 0.1881452000128774
dernières 	 0.1394325430711854
dernières années 	 0.16162551760604463
deux 	 0.07613433035681948
deux dernières 	 0.18677426169052924
données 2014 	 0.18366308310920973
données présente 	 0.1264063957998325
jeu données 	 0.07184316511488423
maladie 	 0.15946663823616855
molécule 	 0.22427292520861608
montants 	 0.15638124740473458
montants remboursés 	 0.21596267378185366
médecine 	 0.17271236061833073
médecine ville 	 0.23211246911817562
présente 	 0.09112539276357223
présente évolution 	 0.1881452000128774
remboursés 	 0.20501195129097466
remboursés assurance 	 0.2278645172975488
ville 	 0.08665535112819407
évolution 